In [4]:
from sedona.spark import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import time

#Αρχικοποίηση SparkSession με 2 executors με 1 core/2 GB memory
spark = SparkSession.builder \
    .appName("Query 4 - Victim Descent") \
    .config("spark.executor.instances", "2") \
    .config("spark.executor.cores", "1") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

#Χρόνος εκτέλεσης για 2 executors με 1 core/2 GB memory
start_time=time.time()

#Δημιουργία sedona context για χρήση geospatial δεδομένων
sedona = SedonaContext.create(spark)

#Διάβασμα του .geojson αρχείου από s3
geojson_path = "s3://initial-notebook-data-bucket-dblab-905418150721/2010_Census_Blocks.geojson"
blocks_df = sedona.read.format("geojson") \
            .option("multiLine", "true").load(geojson_path) \
            .selectExpr("explode(features) as features") \
            .select("features.*")
#Formatting magic
flattened_df = blocks_df.select( \
                [col(f"properties.{col_name}").alias(col_name) for col_name in \
                blocks_df.schema["properties"].dataType.fieldNames()] + ["geometry"]) \
            .drop("properties") \
            .drop("type")

#Φιλτράρισμα για περιοχές του Los Angeles
LA_areas = flattened_df.filter(col("CITY") == "Los Angeles") \
                .groupBy("COMM", "ZCTA10", "POP_2010", "HOUSING10") \
                .agg(ST_Union_Aggr("geometry").alias("geometry"))
                

#Φόρτωση δεδομένων εισοδήματος από το αρχείο CSV
file_path = 's3://initial-notebook-data-bucket-dblab-905418150721/LA_income_2015.csv'  
income_df = spark.read.csv(file_path, header=True, inferSchema=True)

#Μετατροπή του εισοδήματος από string της μορφής "$33,887" σε double της μορφής 33887.0
income_df = income_df.withColumn("Estimated Median Income", 
                                 regexp_replace(substring(col("Estimated Median Income"), 2, 100), ",", "").cast("double"))
income_df = income_df.withColumn("ZipCode", col("Zip Code").cast("double"))

#Μετατροπή του ZCTA10 των περιοχών του Los Angeles από string σε double
LA_areas = LA_areas.withColumn("ZCTA10", col("ZCTA10").cast("double"))


#Join μεταξύ των περιοχών του Los Angeles και των δεδομένων εισοδήματος με βάση το ZCTA10
joined_df = LA_areas.join(income_df,
    LA_areas.ZCTA10 == income_df.ZipCode,"inner").select("COMM", "ZCTA10", "POP_2010", "HOUSING10", *income_df.columns)


#Υπολογισμός συνολικού πληθυσμού και συνολικού αριθμού νοικοκυριών ανά ZIP Code
zip_totals = joined_df.groupBy("ZCTA10").agg(
    sum("POP_2010").alias("TOTAL_ZIP_POP"),
    sum("HOUSING10").alias("TOTAL_ZIP_HOUSING"))

#Join των συνολικών δεδομένων (πληθυσμού και κατοικιών) με τα δεδομένα του κύριου DataFrame
joined_with_totals = joined_df.join(zip_totals,"ZCTA10","left")

#Ομαδοποίηση ανά περιοχή (COMM) και υπολογισμός εκτιμώμενου εισοδήματος ανά άτομο
result_df = joined_with_totals.groupBy("COMM").agg(
    sum("TOTAL_ZIP_HOUSING").alias("TOTAL_HOUSING"),
    sum("TOTAL_ZIP_POP").alias("TOTAL_POP"),
    avg("Estimated Median Income").alias("AVG_MEDIAN_INCOME")).withColumn("Estimated_Income_Per_Person",
    (col("AVG_MEDIAN_INCOME") * col("TOTAL_HOUSING")) / col("TOTAL_POP"))

#Φιλτράρισμα για να αφαιρεθούν οι περιοχές με κενά δεδομένα (NULL)
result_df = result_df.filter(result_df.Estimated_Income_Per_Person.isNotNull())

#Ταξινόμηση περιοχών με βάση το εισόδημα ανά άτομο (φθίνουσα και αύξουσα σειρά)
desc_df = result_df.orderBy(col("Estimated_Income_Per_Person").desc())
asc_df = result_df.orderBy(col("Estimated_Income_Per_Person"))

#Επιλογή των 3 περιοχών με το υψηλότερο και χαμηλότερο εισόδημα ανά άτομο
top3_df = desc_df.limit(3)
last3_df = asc_df.limit(3)

#Δυναμική εξαγωγή των ονομάτων περιοχών για τις top3 και last3 κοινότητες
top_communities = [row["COMM"] for row in top3_df.select("COMM").distinct().collect()]
last_communities = [row["COMM"] for row in last3_df.select("COMM").distinct().collect()]

#Φόρτωση δεδομένων εγκλημάτων από αρχείο CSV
file_path = 's3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv'  
crimeData_df = spark.read.csv(file_path, header=True, inferSchema=True)

#Προσθήκη στήλης για το έτος και φιλτράρισμα εγκλημάτων που έγιναν το 2015
crimeData_df = crimeData_df.withColumn("year", year(to_date("Date Rptd", "MM/dd/yyyy hh:mm:ss a")))
crimeData_df = crimeData_df.filter(col("year") == "2015")

#Δημιουργία γεωμετρικής στήλης από τα πεδία (LAT, LON)
crimeData_df = crimeData_df.withColumn("geom", ST_Point("LON", "LAT"))

#Spatial join δεδομένων εγκλημάτων με τις περιοχές του Los Angeles
crimeDataGEO_df = crimeData_df \
    .join(LA_areas, ST_Within(crimeData_df.geom, LA_areas.geometry), "inner")


#Προσθήκη στήλης για επισήμανση εγκλημάτων σε top3 κοινότητες
crimeDataTOP_df = crimeDataGEO_df.withColumn("is_top3",F.when(F.col("COMM").isin(top_communities), 1).otherwise(0))

#Προσθήκη στήλης για επισήμανση εγκλημάτων σε last3 κοινότητες
crimeDataLAST_df = crimeDataGEO_df.withColumn("is_last3",F.when(F.col("COMM").isin(last_communities), 1).otherwise(0))

#Επιλογή δεδομένων εγκλημάτων για top3 και last3 κοινότητες
top3 = crimeDataTOP_df.select("COMM", "is_top3", "Vict Descent").where(crimeDataTOP_df.is_top3==1)
last3 = crimeDataLAST_df.select("COMM", "is_last3", "Vict Descent").where(crimeDataLAST_df.is_last3==1)

#Φιλτράρισμα εγκλημάτων με μη διαθέσιμη κατηγορία 
top3 = top3.filter(col("Vict Descent").isNotNull())
last3 = last3.filter(col("Vict Descent").isNotNull())

#Φόρτωση κωδικών κατηγοριοποίησης εγκλημάτων από CSV
vict_path = 's3://initial-notebook-data-bucket-dblab-905418150721/RE_codes.csv'
mapping_df = spark.read.csv(vict_path, header=True)

#Δημιουργία dictionary για την αντιστοίχιση των κωδικών
mapping_dict = {row['Vict Descent']: row['Vict Descent Full'] for row in mapping_df.collect()}

#Μετατροπή των κωδικών "Vict Descent" στις αντίστοιχες περιγραφές
top3 = top3.withColumn(
    "Victim Descent",
    when((col("Vict Descent").isin(list(mapping_dict.keys()))), col("Vict Descent"))
    .otherwise("Error")
)

for code, description in mapping_dict.items():
    top3 = top3.withColumn("Victim Descent",when(col("Vict Descent") == code, description).otherwise(col("Victim Descent")))

#Καταμέτρηση εγκλημάτων ανά κατηγορία (Victim Descent) στις top3 κοινότητες
crimeVictsTop3 = top3.groupBy("Victim Descent").count().withColumnRenamed("count", "Victim Count")

#Επανάληψη της διαδικασίας για τις last3 κοινότητες
last3 = last3.withColumn(
    "Victim Descent",
    when((col("Vict Descent").isin(list(mapping_dict.keys()))), col("Vict Descent"))
    .otherwise("Error") 
)

for code, description in mapping_dict.items():
    last3 = last3.withColumn(
        "Victim Descent",
        when(col("Vict Descent") == code, description).otherwise(col("Victim Descent"))
    )

crimeVictsLast3 = last3.groupBy("Victim Descent").count().withColumnRenamed("count", "Victim Count")

#Μέτρηση του χρόνου εκτέλεσης
end_time=time.time()

#Εμφάνιση των αποτελεσμάτων
print(f"Execution time with 2 executors with 1 core and 2GB memory: {end_time - start_time} seconds")
crimeVictsTop3.orderBy(col("Victim Count").desc()).show(crimeVictsTop3.count(), truncate=False)
crimeVictsLast3.orderBy(col("Victim Count").desc()).show(crimeVictsLast3.count(), truncate=False)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Execution time with 2 executors with 1 core and 2GB memory: 47.18118739128113 seconds
+------------------------------+------------+
|Victim Descent                |Victim Count|
+------------------------------+------------+
|White                         |904         |
|Other                         |190         |
|Hispanic/Latin/Mexican        |93          |
|Black                         |55          |
|Unknown                       |53          |
|Other Asian                   |34          |
|Chinese                       |1           |
|American Indian/Alaskan Native|1           |
+------------------------------+------------+

+------------------------------+------------+
|Victim Descent                |Victim Count|
+------------------------------+------------+
|Hispanic/Latin/Mexican        |3167        |
|Black                         |886         |
|White                         |425         |
|Other                         |263         |
|Other Asian                   |137    

In [5]:
from sedona.spark import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import time

#Αρχικοποίηση SparkSession με 2 executors με 2 cores/4 GB memory
spark = SparkSession.builder \
    .appName("Query 4 - Victim Descent") \
    .config("spark.executor.instances", "2") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

#Χρόνος εκτέλεσης για 2 executors με 2 cores/4 GB memory
start_time=time.time()

#Δημιουργία sedona context για χρήση geospatial δεδομένων
sedona = SedonaContext.create(spark)

#Διάβασμα του .geojson αρχείου από s3
geojson_path = "s3://initial-notebook-data-bucket-dblab-905418150721/2010_Census_Blocks.geojson"
blocks_df = sedona.read.format("geojson") \
            .option("multiLine", "true").load(geojson_path) \
            .selectExpr("explode(features) as features") \
            .select("features.*")
#Formatting magic
flattened_df = blocks_df.select( \
                [col(f"properties.{col_name}").alias(col_name) for col_name in \
                blocks_df.schema["properties"].dataType.fieldNames()] + ["geometry"]) \
            .drop("properties") \
            .drop("type")

#Φιλτράρισμα για περιοχές του Los Angeles
LA_areas = flattened_df.filter(col("CITY") == "Los Angeles") \
                .groupBy("COMM", "ZCTA10", "POP_2010", "HOUSING10") \
                .agg(ST_Union_Aggr("geometry").alias("geometry"))
                

#Φόρτωση δεδομένων εισοδήματος από το αρχείο CSV
file_path = 's3://initial-notebook-data-bucket-dblab-905418150721/LA_income_2015.csv'  
income_df = spark.read.csv(file_path, header=True, inferSchema=True)

#Μετατροπή του εισοδήματος από string της μορφής "$33,887" σε double της μορφής 33887.0
income_df = income_df.withColumn("Estimated Median Income", 
                                 regexp_replace(substring(col("Estimated Median Income"), 2, 100), ",", "").cast("double"))
income_df = income_df.withColumn("ZipCode", col("Zip Code").cast("double"))

#Μετατροπή του ZCTA10 των περιοχών του Los Angeles από string σε double
LA_areas = LA_areas.withColumn("ZCTA10", col("ZCTA10").cast("double"))


#Join μεταξύ των περιοχών του Los Angeles και των δεδομένων εισοδήματος με βάση το ZCTA10
joined_df = LA_areas.join(income_df,
    LA_areas.ZCTA10 == income_df.ZipCode,"inner").select("COMM", "ZCTA10", "POP_2010", "HOUSING10", *income_df.columns)


#Υπολογισμός συνολικού πληθυσμού και συνολικού αριθμού νοικοκυριών ανά ZIP Code
zip_totals = joined_df.groupBy("ZCTA10").agg(
    sum("POP_2010").alias("TOTAL_ZIP_POP"),
    sum("HOUSING10").alias("TOTAL_ZIP_HOUSING"))

#Join των συνολικών δεδομένων (πληθυσμού και κατοικιών) με τα δεδομένα του κύριου DataFrame
joined_with_totals = joined_df.join(zip_totals,"ZCTA10","left")

#Ομαδοποίηση ανά περιοχή (COMM) και υπολογισμός εκτιμώμενου εισοδήματος ανά άτομο
result_df = joined_with_totals.groupBy("COMM").agg(
    sum("TOTAL_ZIP_HOUSING").alias("TOTAL_HOUSING"),
    sum("TOTAL_ZIP_POP").alias("TOTAL_POP"),
    avg("Estimated Median Income").alias("AVG_MEDIAN_INCOME")).withColumn("Estimated_Income_Per_Person",
    (col("AVG_MEDIAN_INCOME") * col("TOTAL_HOUSING")) / col("TOTAL_POP"))

#Φιλτράρισμα για να αφαιρεθούν οι περιοχές με κενά δεδομένα (NULL)
result_df = result_df.filter(result_df.Estimated_Income_Per_Person.isNotNull())

#Ταξινόμηση περιοχών με βάση το εισόδημα ανά άτομο (φθίνουσα και αύξουσα σειρά)
desc_df = result_df.orderBy(col("Estimated_Income_Per_Person").desc())
asc_df = result_df.orderBy(col("Estimated_Income_Per_Person"))

#Επιλογή των 3 περιοχών με το υψηλότερο και χαμηλότερο εισόδημα ανά άτομο
top3_df = desc_df.limit(3)
last3_df = asc_df.limit(3)

#Δυναμική εξαγωγή των ονομάτων περιοχών για τις top3 και last3 κοινότητες
top_communities = [row["COMM"] for row in top3_df.select("COMM").distinct().collect()]
last_communities = [row["COMM"] for row in last3_df.select("COMM").distinct().collect()]

#Φόρτωση δεδομένων εγκλημάτων από αρχείο CSV
file_path = 's3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv'  
crimeData_df = spark.read.csv(file_path, header=True, inferSchema=True)

#Προσθήκη στήλης για το έτος και φιλτράρισμα εγκλημάτων που έγιναν το 2015
crimeData_df = crimeData_df.withColumn("year", year(to_date("Date Rptd", "MM/dd/yyyy hh:mm:ss a")))
crimeData_df = crimeData_df.filter(col("year") == "2015")

#Δημιουργία γεωμετρικής στήλης από τα πεδία (LAT, LON)
crimeData_df = crimeData_df.withColumn("geom", ST_Point("LON", "LAT"))

#Spatial join δεδομένων εγκλημάτων με τις περιοχές του Los Angeles
crimeDataGEO_df = crimeData_df \
    .join(LA_areas, ST_Within(crimeData_df.geom, LA_areas.geometry), "inner")


#Προσθήκη στήλης για επισήμανση εγκλημάτων σε top3 κοινότητες
crimeDataTOP_df = crimeDataGEO_df.withColumn("is_top3",F.when(F.col("COMM").isin(top_communities), 1).otherwise(0))

#Προσθήκη στήλης για επισήμανση εγκλημάτων σε last3 κοινότητες
crimeDataLAST_df = crimeDataGEO_df.withColumn("is_last3",F.when(F.col("COMM").isin(last_communities), 1).otherwise(0))

#Επιλογή δεδομένων εγκλημάτων για top3 και last3 κοινότητες
top3 = crimeDataTOP_df.select("COMM", "is_top3", "Vict Descent").where(crimeDataTOP_df.is_top3==1)
last3 = crimeDataLAST_df.select("COMM", "is_last3", "Vict Descent").where(crimeDataLAST_df.is_last3==1)

#Φιλτράρισμα εγκλημάτων με μη διαθέσιμη κατηγορία 
top3 = top3.filter(col("Vict Descent").isNotNull())
last3 = last3.filter(col("Vict Descent").isNotNull())

#Φόρτωση κωδικών κατηγοριοποίησης εγκλημάτων από CSV
vict_path = 's3://initial-notebook-data-bucket-dblab-905418150721/RE_codes.csv'
mapping_df = spark.read.csv(vict_path, header=True)

#Δημιουργία dictionary για την αντιστοίχιση των κωδικών
mapping_dict = {row['Vict Descent']: row['Vict Descent Full'] for row in mapping_df.collect()}

#Μετατροπή των κωδικών "Vict Descent" στις αντίστοιχες περιγραφές
top3 = top3.withColumn(
    "Victim Descent",
    when((col("Vict Descent").isin(list(mapping_dict.keys()))), col("Vict Descent"))
    .otherwise("Error")
)

for code, description in mapping_dict.items():
    top3 = top3.withColumn("Victim Descent",when(col("Vict Descent") == code, description).otherwise(col("Victim Descent")))

#Καταμέτρηση εγκλημάτων ανά κατηγορία (Victim Descent) στις top3 κοινότητες
crimeVictsTop3 = top3.groupBy("Victim Descent").count().withColumnRenamed("count", "Victim Count")

#Επανάληψη της διαδικασίας για τις last3 κοινότητες
last3 = last3.withColumn(
    "Victim Descent",
    when((col("Vict Descent").isin(list(mapping_dict.keys()))), col("Vict Descent"))
    .otherwise("Error") 
)

for code, description in mapping_dict.items():
    last3 = last3.withColumn(
        "Victim Descent",
        when(col("Vict Descent") == code, description).otherwise(col("Victim Descent"))
    )

crimeVictsLast3 = last3.groupBy("Victim Descent").count().withColumnRenamed("count", "Victim Count")

#Μέτρηση του χρόνου εκτέλεσης
end_time=time.time()

#Εμφάνιση των αποτελεσμάτων
print(f"Execution time with 2 executors with 2 cores and 4GB memory: {end_time - start_time} seconds")
crimeVictsTop3.orderBy(col("Victim Count").desc()).show(crimeVictsTop3.count(), truncate=False)
crimeVictsLast3.orderBy(col("Victim Count").desc()).show(crimeVictsLast3.count(), truncate=False)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Execution time with 2 executors with 2 cores and 4GB memory: 28.503281831741333 seconds
+------------------------------+------------+
|Victim Descent                |Victim Count|
+------------------------------+------------+
|White                         |904         |
|Other                         |190         |
|Hispanic/Latin/Mexican        |93          |
|Black                         |55          |
|Unknown                       |53          |
|Other Asian                   |34          |
|Chinese                       |1           |
|American Indian/Alaskan Native|1           |
+------------------------------+------------+

+------------------------------+------------+
|Victim Descent                |Victim Count|
+------------------------------+------------+
|Hispanic/Latin/Mexican        |3167        |
|Black                         |886         |
|White                         |425         |
|Other                         |263         |
|Other Asian                   |137  

In [8]:
from sedona.spark import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import time

#Αρχικοποίηση SparkSession με 2 executors με 4 cores/8 GB memory
spark = SparkSession.builder \
    .appName("Query 4 - Victim Descent") \
    .config("spark.executor.instances", "2") \
    .config("spark.executor.cores", "4") \
    .config("spark.executor.memory", "8g") \
    .getOrCreate()

#Χρόνος εκτέλεσης για 2 executors με 4 cores/8 GB memory
start_time=time.time()

#Δημιουργία sedona context για χρήση geospatial δεδομένων
sedona = SedonaContext.create(spark)

#Διάβασμα του .geojson αρχείου από s3
geojson_path = "s3://initial-notebook-data-bucket-dblab-905418150721/2010_Census_Blocks.geojson"
blocks_df = sedona.read.format("geojson") \
            .option("multiLine", "true").load(geojson_path) \
            .selectExpr("explode(features) as features") \
            .select("features.*")
#Formatting magic
flattened_df = blocks_df.select( \
                [col(f"properties.{col_name}").alias(col_name) for col_name in \
                blocks_df.schema["properties"].dataType.fieldNames()] + ["geometry"]) \
            .drop("properties") \
            .drop("type")

#Φιλτράρισμα για περιοχές του Los Angeles
LA_areas = flattened_df.filter(col("CITY") == "Los Angeles") \
                .groupBy("COMM", "ZCTA10", "POP_2010", "HOUSING10") \
                .agg(ST_Union_Aggr("geometry").alias("geometry"))
                

#Φόρτωση δεδομένων εισοδήματος από το αρχείο CSV
file_path = 's3://initial-notebook-data-bucket-dblab-905418150721/LA_income_2015.csv'  
income_df = spark.read.csv(file_path, header=True, inferSchema=True)

#Μετατροπή του εισοδήματος από string της μορφής "$33,887" σε double της μορφής 33887.0
income_df = income_df.withColumn("Estimated Median Income", 
                                 regexp_replace(substring(col("Estimated Median Income"), 2, 100), ",", "").cast("double"))
income_df = income_df.withColumn("ZipCode", col("Zip Code").cast("double"))

#Μετατροπή του ZCTA10 των περιοχών του Los Angeles από string σε double
LA_areas = LA_areas.withColumn("ZCTA10", col("ZCTA10").cast("double"))


#Join μεταξύ των περιοχών του Los Angeles και των δεδομένων εισοδήματος με βάση το ZCTA10
joined_df = LA_areas.join(income_df,
    LA_areas.ZCTA10 == income_df.ZipCode,"inner").select("COMM", "ZCTA10", "POP_2010", "HOUSING10", *income_df.columns)


#Υπολογισμός συνολικού πληθυσμού και συνολικού αριθμού νοικοκυριών ανά ZIP Code
zip_totals = joined_df.groupBy("ZCTA10").agg(
    sum("POP_2010").alias("TOTAL_ZIP_POP"),
    sum("HOUSING10").alias("TOTAL_ZIP_HOUSING"))

#Join των συνολικών δεδομένων (πληθυσμού και κατοικιών) με τα δεδομένα του κύριου DataFrame
joined_with_totals = joined_df.join(zip_totals,"ZCTA10","left")

#Ομαδοποίηση ανά περιοχή (COMM) και υπολογισμός εκτιμώμενου εισοδήματος ανά άτομο
result_df = joined_with_totals.groupBy("COMM").agg(
    sum("TOTAL_ZIP_HOUSING").alias("TOTAL_HOUSING"),
    sum("TOTAL_ZIP_POP").alias("TOTAL_POP"),
    avg("Estimated Median Income").alias("AVG_MEDIAN_INCOME")).withColumn("Estimated_Income_Per_Person",
    (col("AVG_MEDIAN_INCOME") * col("TOTAL_HOUSING")) / col("TOTAL_POP"))

#Φιλτράρισμα για να αφαιρεθούν οι περιοχές με κενά δεδομένα (NULL)
result_df = result_df.filter(result_df.Estimated_Income_Per_Person.isNotNull())

#Ταξινόμηση περιοχών με βάση το εισόδημα ανά άτομο (φθίνουσα και αύξουσα σειρά)
desc_df = result_df.orderBy(col("Estimated_Income_Per_Person").desc())
asc_df = result_df.orderBy(col("Estimated_Income_Per_Person"))

#Επιλογή των 3 περιοχών με το υψηλότερο και χαμηλότερο εισόδημα ανά άτομο
top3_df = desc_df.limit(3)
last3_df = asc_df.limit(3)

#Δυναμική εξαγωγή των ονομάτων περιοχών για τις top3 και last3 κοινότητες
top_communities = [row["COMM"] for row in top3_df.select("COMM").distinct().collect()]
last_communities = [row["COMM"] for row in last3_df.select("COMM").distinct().collect()]

#Φόρτωση δεδομένων εγκλημάτων από αρχείο CSV
file_path = 's3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv'  
crimeData_df = spark.read.csv(file_path, header=True, inferSchema=True)

#Προσθήκη στήλης για το έτος και φιλτράρισμα εγκλημάτων που έγιναν το 2015
crimeData_df = crimeData_df.withColumn("year", year(to_date("Date Rptd", "MM/dd/yyyy hh:mm:ss a")))
crimeData_df = crimeData_df.filter(col("year") == "2015")

#Δημιουργία γεωμετρικής στήλης από τα πεδία (LAT, LON)
crimeData_df = crimeData_df.withColumn("geom", ST_Point("LON", "LAT"))

#Spatial join δεδομένων εγκλημάτων με τις περιοχές του Los Angeles
crimeDataGEO_df = crimeData_df \
    .join(LA_areas, ST_Within(crimeData_df.geom, LA_areas.geometry), "inner")


#Προσθήκη στήλης για επισήμανση εγκλημάτων σε top3 κοινότητες
crimeDataTOP_df = crimeDataGEO_df.withColumn("is_top3",F.when(F.col("COMM").isin(top_communities), 1).otherwise(0))

#Προσθήκη στήλης για επισήμανση εγκλημάτων σε last3 κοινότητες
crimeDataLAST_df = crimeDataGEO_df.withColumn("is_last3",F.when(F.col("COMM").isin(last_communities), 1).otherwise(0))

#Επιλογή δεδομένων εγκλημάτων για top3 και last3 κοινότητες
top3 = crimeDataTOP_df.select("COMM", "is_top3", "Vict Descent").where(crimeDataTOP_df.is_top3==1)
last3 = crimeDataLAST_df.select("COMM", "is_last3", "Vict Descent").where(crimeDataLAST_df.is_last3==1)

#Φιλτράρισμα εγκλημάτων με μη διαθέσιμη κατηγορία 
top3 = top3.filter(col("Vict Descent").isNotNull())
last3 = last3.filter(col("Vict Descent").isNotNull())

#Φόρτωση κωδικών κατηγοριοποίησης εγκλημάτων από CSV
vict_path = 's3://initial-notebook-data-bucket-dblab-905418150721/RE_codes.csv'
mapping_df = spark.read.csv(vict_path, header=True)

#Δημιουργία dictionary για την αντιστοίχιση των κωδικών
mapping_dict = {row['Vict Descent']: row['Vict Descent Full'] for row in mapping_df.collect()}

#Μετατροπή των κωδικών "Vict Descent" στις αντίστοιχες περιγραφές
top3 = top3.withColumn(
    "Victim Descent",
    when((col("Vict Descent").isin(list(mapping_dict.keys()))), col("Vict Descent"))
    .otherwise("Error")
)

for code, description in mapping_dict.items():
    top3 = top3.withColumn("Victim Descent",when(col("Vict Descent") == code, description).otherwise(col("Victim Descent")))

#Καταμέτρηση εγκλημάτων ανά κατηγορία (Victim Descent) στις top3 κοινότητες
crimeVictsTop3 = top3.groupBy("Victim Descent").count().withColumnRenamed("count", "Victim Count")

#Επανάληψη της διαδικασίας για τις last3 κοινότητες
last3 = last3.withColumn(
    "Victim Descent",
    when((col("Vict Descent").isin(list(mapping_dict.keys()))), col("Vict Descent"))
    .otherwise("Error") 
)

for code, description in mapping_dict.items():
    last3 = last3.withColumn(
        "Victim Descent",
        when(col("Vict Descent") == code, description).otherwise(col("Victim Descent"))
    )

crimeVictsLast3 = last3.groupBy("Victim Descent").count().withColumnRenamed("count", "Victim Count")

#Μέτρηση του χρόνου εκτέλεσης
end_time=time.time()

#Εμφάνιση των αποτελεσμάτων
print(f"Execution time with 2 executors with 4 cores and 8GB memory: {end_time - start_time} seconds")
crimeVictsTop3.orderBy(col("Victim Count").desc()).show(crimeVictsTop3.count(), truncate=False)
crimeVictsLast3.orderBy(col("Victim Count").desc()).show(crimeVictsLast3.count(), truncate=False)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Execution time with 2 executors with 4 cores and 8GB memory: 44.97376775741577 seconds
+------------------------------+------------+
|Victim Descent                |Victim Count|
+------------------------------+------------+
|White                         |904         |
|Other                         |190         |
|Hispanic/Latin/Mexican        |93          |
|Black                         |55          |
|Unknown                       |53          |
|Other Asian                   |34          |
|American Indian/Alaskan Native|1           |
|Chinese                       |1           |
+------------------------------+------------+

+------------------------------+------------+
|Victim Descent                |Victim Count|
+------------------------------+------------+
|Hispanic/Latin/Mexican        |3167        |
|Black                         |886         |
|White                         |425         |
|Other                         |263         |
|Other Asian                   |137   